# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install python-dotenv

In [ ]:
import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

# OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY3')

from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY3')

HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
#!pip install pandas

In [ ]:
#import file from colab

from google.colab import drive
drive.mount('/content/drive')

# Path to your CSV file in Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/data/Data.csv'

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv(file_path)



In [ ]:
df.head()

,Product,Review,Summary,Translated_Summary_Dutch,Review_sum_dutch
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...,None,None,None
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi...",None,None,None
2,Luxury Air Mattress,This mattress had a small hole in the top of i...,None,None,None
3,Pillows Insert,This is the best throw pillow fillers on Amazo...,None,None,None
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...,None,None,None


In [ ]:
df = pd.DataFrame(data)
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain

In [ ]:
#Replace None by your own value and justify
#temperature determines the ramdomness of the output. 1 is neutral
llm = ChatOpenAI(model_name="gpt-3.5-turbo-instruct", api_key=api_key, temperature=1)


In [ ]:
!pip install langchain_community
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(api_key=api_key,temperature=0.9)

# from template can handle multiple sentenses
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)

product = "Tesla"
output = chain.run(product)
print(output)

Electric Innovations Co.


## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(api_key=api_key,temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
      "What is the best name to describe \
      a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
output = overall_simple_chain.run("Tesla")
print(output)




> Entering new SimpleSequentialChain chain...
Electra Motors
Electra Motors is a leading manufacturer of electric vehicles, specializing in innovative designs and sustainable transportation solutions for the future.

> Finished chain.
Electra Motors is a leading manufacturer of electric vehicles, specializing in innovative designs and sustainable transportation solutions for the future.


**Repeat the above twice for different products**

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:

llm = ChatOpenAI(api_key=api_key, temperature=0.9)

# Function to process each review
def process_review(review_text):

    # translate to French
    first_prompt = ChatPromptTemplate.from_template(
      "Translate the text to French {review}"
    )

    chain_one = LLMChain(llm=llm, prompt=first_prompt,
                        output_key="translated_review"
                        )

    second_prompt = ChatPromptTemplate.from_template(
      "Summarize in 20 words the following review in English: {translated_review}"
    )

    chain_two = LLMChain(llm=llm, prompt=second_prompt,
                      output_key="summary_review"
                      )

    # prompt template 3: translate to Dutch
    third_prompt = ChatPromptTemplate.from_template(
        "Translate the {summary_review} to Dutch"
    )
    # chain 3: input= Review and output= language
    chain_three = LLMChain(llm=llm, prompt=third_prompt,
                          output_key="translated_dutch"
                          )

    # overall_chain: input= Review
    # and output= English_Review,summary, followup_message
    overall_chain = SequentialChain(
      chains=[chain_one, chain_two, chain_three],
      input_variables=["review"],
      output_variables=["translated_dutch"],
      verbose=True
    )

    # Run the chain with the input review
    output = overall_chain.run({"review": review_text})
    return output


In [ ]:
# Apply the process to each review in the DataFrame
df['Review_sum_dutch'] = df['Review'].apply(process_review)

# Print the resulting DataFrame
df.head()



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


,Product,Review,Review_sum_dutch
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...,Vertaalde koning size set gekocht. Wilde 4 kus...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi...","Hield van de waterdichte tas, maar de harde pl..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...,De matras had een klein gaatje aan de bovenkan...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...,"De beste kussen inserts op Amazon, ik heb ande..."
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...,Hield van het product maar het heeft maar een ...


In [ ]:
review = df.Review[5]
# overall_chain(Review)

print(review)

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?


**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(api_key=api_key, temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [ ]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [ ]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**